In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import cv2
import os

_*OpenCV란?*_

- Computer Vision library 의 약자로 이미지 검수를 기계적으로 처리하도록 도와주는 이미지 처리 라이브러리

- Image Processing : Circle Detection, 이미지 선명하게 만들기, 필터를 이용해서 edge detection 등에 응용

- Robot/ Machine / Video / Vision : 얼굴인식, 무인자동차가 장애물 인식, 사람 객체 인식, 이미지 매칭 등에 응용

- AI : 딥러닝에 사용, 이미지가 어떤 내용인지 이해, 이미지의 얼굴 감정 인지 등에 응용가능

- 3D Geometry (기하학) : 이미지에서 3차원 정보를 인식할 수 있다. 여러 측면의 이미지를 조합해서 3D 도면 구성 가능 이와 같이 다양한 분야에서 응용될 수 있는 이미지 처리 라이브러리!!

- 영상 처리와 컴퓨터 비전 관련 오픈소스 라이브러리로 250개가 넘는 알고리즘으로 구성됨

In [3]:
Image_width,Image_height = 400,400
batch_size=50
val_split=0.01
n_train=25000*(1-val_split) # train data는 총 25000개 중에서 99%만큼
n_val=25000*val_split # validation data는 총 35000개 중에서 1%만큼

- 메모리의 한계와 속도 저하 때문에 대부분의 경우에는 한 번의 epoch에서 모든 데이터를 한꺼번에 집어넣을 수는 없음. 그래서 데이터를 나누어서 주게 되는데 이때 몇 번 나누어서 주는가를 iteration, 각 iteration마다 주는 데이터 사이즈를 batch size라고 함.

In [4]:
train = pd.DataFrame({"name" :os.listdir("C:/Users/YANG/Desktop/image/train"), 
                      "path" :pd.Series(os.listdir("C:/Users/YANG/Desktop/image/train")).apply(lambda x : "C:/Users/YANG/Desktop/image/train/" + str(x)), 
                      "label" :pd.Series(os.listdir("C:/Users/YANG/Desktop/image/train")).apply(lambda x : str(x).split(".")[0])})
# train dataset의 name/path/label을 데이터프레임으로 저장

In [5]:
train.head()

,name,path,label
0,cat.0.jpg,C:/Users/YANG/Desktop/image/train/cat.0.jpg,cat
1,cat.1.jpg,C:/Users/YANG/Desktop/image/train/cat.1.jpg,cat
2,cat.10.jpg,C:/Users/YANG/Desktop/image/train/cat.10.jpg,cat
3,cat.100.jpg,C:/Users/YANG/Desktop/image/train/cat.100.jpg,cat
4,cat.1000.jpg,C:/Users/YANG/Desktop/image/train/cat.1000.jpg,cat


In [6]:
test = pd.DataFrame({"name" :os.listdir("C:/Users/YANG/Desktop/image/test"), 
                      "path" :pd.Series(os.listdir("C:/Users/YANG/Desktop/image/test")).apply(lambda x : "C:/Users/YANG/Desktop/image/test/" + str(x)), 
                     })

In [7]:
test.head()

,name,path
0,1.jpg,C:/Users/YANG/Desktop/image/test/1.jpg
1,10.jpg,C:/Users/YANG/Desktop/image/test/10.jpg
2,100.jpg,C:/Users/YANG/Desktop/image/test/100.jpg
3,1000.jpg,C:/Users/YANG/Desktop/image/test/1000.jpg
4,10000.jpg,C:/Users/YANG/Desktop/image/test/10000.jpg


In [8]:
from keras.preprocessing.image import ImageDataGenerator

train_image_gen = ImageDataGenerator(rescale=1/255, horizontal_flip=True, validation_split=val_split)

Using TensorFlow backend.


train_generator = train_image_gen.flow_from_dataframe(dataframe = train, x_col = "path", y_col ="label",weight_col = None,
                                                      target_size=(Image_width,Image_height), 
                                                      batch_size=batch_size,seed=19,subset='training',
                                                      shuffle=True, class_mode='categorical')

val_generator = train_image_gen.flow_from_dataframe(dataframe = train, x_col = "path", y_col ="label",weight_col = None,
                                                    target_size=(Image_width,Image_height), 
                                                    batch_size=batch_size,seed=19,subset='validation',
                                                    shuffle=True, class_mode='categorical')

## *ImageDataGenerator*
- 이미지를 사용할 때마다 임의로 변형을 가함으로써 마치 훨씬 더 많은 이미지를 보고 공부하는 것과 같은 학습 효과를 내기 위해 image augmentation을 실행.  
이를 통해 과적합 (overfitting), 즉 모델이 학습 데이터에만 맞춰지는 것을 방지하고, 새로운 이미지도 잘 분류할 수 있게 됨.  

_옵션에 대한 부가설명_
1. rescale: 원본 영상은 0-255의 RGB 계수로 구성되는데, 이 같은 입력값은 모델을 효과적으로 학습시키기에 너무 높다. (통상적인 learning rate를 사용할 경우). 따라서 이를 1/255로 스케일링하여 0-1 범위로 변환시켜준다. 이는 다른 전처리 과정에 앞서 가장 먼저 적용됨.
2. horizontal_flip: True로 설정할 경우, 50% 확률로 이미지를 수평으로 뒤집는다. 원본 이미지에 수평 비대칭성이 없을 때 효과적

In [28]:
from keras.optimizers import SGD
from keras.layers import Dense, Dropout
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
# Dropout 알아보기... overfitting 방지하기 위해 사용. - 이미지에서는 overfitting이 잘 일어나지 않으니까 잘 쓰지 않음.

Instructions for updating:
Colocations handled automatically by placer.
87916544/87910968 [==============================] - 142s 2us/step


In [32]:
# create the base pre-trained model
InceptionV3_base_model = InceptionV3(weights='imagenet', include_top=False, pooling = "avg")
x = InceptionV3_base_model.output

## *inception v3 model*
1. include_top: whether to include the fully-connected layer at the top of the network.
2. weights: one of None (random initialization) or 'imagenet' (pre-training on ImageNet).
3. input_tensor: optional Keras tensor (i.e. output of layers.Input()) to use as image input for the model.
4. input_shape: optional shape tuple, only to be specified if include_top is False (otherwise the input shape has to be (299, 299, 3) (with 'channels_last' data format) or (3, 299, 299) (with 'channels_first' data format). It should have exactly 3 inputs channels, and width and height should be no smaller than 75. E.g. (150, 150, 3) would be one valid value.
5. pooling: Optional pooling mode for feature extraction when include_top is False.
 - None means that the output of the model will be the 4D tensor output of the last convolutional block.
 - 'avg' means that global average pooling will be applied to the output of the last convolutional block, and thus the output of the model will be a 2D tensor.
 - 'max' means that global max pooling will be applied.
6. classes: optional number of classes to classify images into, only to be specified if include_top is True, and if no weights argument is specified.

In [33]:
# add a fully connected layer after Inception
## 은닉층이므로 activation은 relu
x_dense = Dense(1024,activation='relu')(x)

# and add a Logistic Layter -- 다중클래스(2개)로 분류하니까 activation은 softmax
final_pred = Dense(2,activation='softmax')(x_dense)

model = Model(inputs=InceptionV3_base_model.input, outputs=final_pred)

In [ ]:
# 앞에서 불러온 inception v3 모델 형식 에다가 input이랑 output만 넣어주어도 인셉션으로 모델이 돌아감.
# inception v3틀만 가지고 왔는데 여기에다가 Model을 넣어주기만 해도 인셉션 모델대로 돌아가는 것인가 !
# input이랑 output만 지정해줘도 인셉션 모델로 돌아간다..

## *Dense layer*
Dense 레이어는 입력과 출력을 모두 연결해줍니다. 예를 들어 입력 뉴런이 4개, 출력 뉴런이 8개있다면 총 연결선은 32개(4*8=32) 입니다. 각 연결선에는 가중치(weight)를 포함하고 있는데, 이 가중치가 나타내는 의미는 연결강도라고 보시면 됩니다. 현재 연결선이 32개이므로 가중치도 32개입니다.  
(가중치가 높을수록 해당 입력 뉴런이 출력 뉴런에 미치는 영향이 크고, 낮을수록 미치는 영향이 적다.)  

ex. 예를 들어 성별을 판단하는 문제있어서, 출력 뉴런의 값이 성별을 의미하고, 입력 뉴런에 머리카락길이, 키, 혈핵형 등이 있다고 가정했을 때, 머리카락길이의 가중치가 가장 높고, 키의 가중치가 중간이고, 혈핵형의 가중치가 가장 낮을 겁니다. 딥러닝 학습과정에서 이러한 가중치들이 조정됩니다.  

#### option
1. 첫번째 인자 : 출력 뉴런의 수를 설정
2. input_dim : 입력 뉴런의 수를 설정.
3. init : 가중치 초기화 방법 설정
 - 'uniform’ : 균일 분포
 - 'normal’ : 가우시안 분포
4. activation : 활성화 함수 설정
 - 'linear’ : 디폴트 값, 입력뉴런과 가중치로 계산된 결과값이 그대로 출력으로 나옴.
 - 'relu’ : rectifier 함수, 은닉층에 주로 쓰임.
 - 'sigmoid’ : 시그모이드 함수, 이진 분류 문제에서 출력층에 주로 쓰임.
 - 'softmax’ : 소프트맥스 함수, 다중 클래스 분류 문제에서 출력층에 주로 쓰임.   
 (입력 뉴런과 가중치를 계산한 값을 각 클래스의 확률 개념으로 표현할 수 있는 활성화 함수)
 
 [출처] https://tykimos.github.io/2017/01/27/MLP_Layer_Talk/

In [34]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_95 (Conv2D)              (None, None, None, 3 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_95 (BatchNo (None, None, None, 3 96          conv2d_95[0][0]                  
__________________________________________________________________________________________________
activation_95 (Activation)      (None, None, None, 3 0           batch_normalization_95[0][0]     
____________________________________________________________________________________________

In [ ]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('best.hd5', monitor = 'val_loss', save_best_only = True)

### ModelCheckpoint


In [ ]:
layer_to_Freeze=172 
for layer in model.layers[:layer_to_Freeze]:
    layer.trainable = False
for layer in model.layers[layer_to_Freeze:]:
    layer.trainable = True

In [ ]:
sgd = SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(optimizer=sgd, loss='categorical_crossentropy',metrics=['accuracy'])

* model compilation

Before training a model, you need to configure the learning process, which is done via the compile method.  
It receives three arguments:

1. An optimizer. This could be the string identifier of an existing optimizer (such as rmsprop or adagrad), or an instance of the Optimizer class.
2. A loss function. This is the objective that the model will try to minimize. It can be the string identifier of an existing loss function (such as categorical_crossentropy or mse), or it can be an objective function. 
3. A list of metrics. For any classification problem you will want to set this to metrics=['accuracy']. A metric could be the string identifier of an existing metric or a custom metric function.


_optimizer 옵션 중 SGD란?_

-Stochastic gradient descent optimizer.

-Includes support for momentum, learning rate decay, and Nesterov momentum.

-Arguments

1. learning_rate: float >= 0. Learning rate.
2. momentum: float >= 0. Parameter that accelerates SGD in the relevant direction and dampens oscillations.
3. nesterov: boolean. Whether to apply Nesterov momentum.

그 외 optimizer option들에 대한 설명:
https://keras.io/optimizers/

In [ ]:
model.fit_generator(train_generator, epochs=1,
                                                steps_per_epoch=n_train//batch_size,
                                                validation_data=val_generator,
                                                validation_steps=n_val//batch_size,
                                                verbose=1,
                                                callbacks=[checkpoint],)

In [ ]:
model.load_weights('best.hd5')

In [ ]:
test_image_gen = ImageDataGenerator(rescale=1/255)

In [ ]:
test_generator = test_image_gen.flow_from_dataframe(dataframe = test, x_col = "path", y_col = None,
                                                    target_size=(Image_width,Image_height), batch_size=batch_size, seed=42, class_mode=None, shuffle=False)

In [ ]:
y_pred = model.predict_generator(generator=test_generator, steps=int(np.ceil(len(test)/batch_size)), workers = 2, verbose=1)

In [ ]:
y_pred

In [ ]:
submission = pd.DataFrame({'id':pd.Series(test_generator.filenames),'label':pd.Series(y_pred.clip(min=0.02, max=0.98)[:,1])})
submission["id"] = submission["id"].apply(lambda x: str(x).split("/")[-1].split(".")[0])
submission.to_csv('DogVsCats_submission.csv',index=False)
submission.head()